In [4]:
import pandas as pd
import csv
import re

In [1]:
user_name = input("User Name: ")
directory1 = "C:\\Users\\"
directory2 = "\\Desktop\\Payroll System"
directory = directory1 + user_name + directory2

User Name: ASUS


In [2]:
folder_name = str(input("Name of Folder: "))
in_folder_name = "In-" + folder_name

Name of Folder: July 22-28


In [5]:
Employee_Information_df = pd.read_csv(directory + '\\1 Information CSVs\\Employee-Information.csv')
Standard_Clock_Hours_df = pd.read_csv(directory + '\\1 Information CSVs\\Standard-Clock-Hours.csv')

day_name = str(input("Name of Input: "))
in_directory_folder_name = directory + "\\" + folder_name + "\\" + in_folder_name + "\\" + "In-" + day_name + ".csv"

#Input
Day_df = pd.read_csv(in_directory_folder_name)

Name of Input: July 22


In [7]:
def time_to_minutes(x):
    if len(str(x))==3:
        hours = str(x)
        hours = int(hours[0])*60
        minutes = str(x)
        minutes = int(minutes[1:3])
        time = hours + minutes
        time = str(time)
        return time
    if len(str(x))==4:
        hours = str(x)
        hours = int(hours[0:2])*60
        minutes = str(x)
        minutes = int(minutes[2:4])
        time = hours + minutes
        time = str(time)
        return time

In [8]:
def time_round_up(x):
    hours = int(x)/60
    rational = float(hours)
    integer = int(hours)
    decimal = rational - integer
    if decimal >=0.5:
        roundnumber = integer + 1
        return str(roundnumber)
    else:
        return str(integer)

In [9]:
def work_hours(x):
    if int(x)<=8:
        work_hours = int(x)
        return work_hours
    else:
        work_hours = 8
        return work_hours
    return work_hours

In [10]:
def overtime_hours(x):
    if int(x)<=8:
        overtime_hours = 0
        return overtime_hours
    elif int(x)>8:
        overtime_hours = int(x) - 8
        return overtime_hours
    return overtime_hours

In [11]:
def late_hours(x):
    if int(x)<=0:
        late_hours = 0
        return late_hours
    elif int(x)>0:
        late_hours = int(x) / 60
        rational = float(late_hours)
        integer = int(late_hours)
        decimal = rational - integer
        if decimal >=0:
            late_hours = integer + 1
            return late_hours
        else:
            return integer

In [12]:
# Removes Colons of time in Standard Clock Hours and Input CSV
Day_df.loc[:,"Time In"] = Day_df.loc[:,"Time In"].apply(lambda x: re.sub(":","", x) )
Day_df.loc[:,"Time Out"] = Day_df.loc[:,"Time Out"].apply(lambda x: re.sub(":","", x) )

Standard_Clock_Hours_df.loc[:,"Clock In Time"] = Standard_Clock_Hours_df.loc[:,"Clock In Time"].apply(lambda x: re.sub(":","", x) )
Standard_Clock_Hours_df.loc[:,"Clock Out Time"] = Standard_Clock_Hours_df.loc[:,"Clock Out Time"].apply(lambda x: re.sub(":","", x) )

In [13]:
def debt_tracker(x):
    paid_debt = 0
    ID_Number = x
    debt_file_name = "DebtTracker_" + str(ID_Number) + ".csv"
    temp_df = pd.read_csv(directory + '\\3 Debt Info\\' + debt_file_name)
    Debt_Tracker_df = temp_df.loc[:,"Debt Number":]
    
    for i in range(0,len(Debt_Tracker_df.loc[:,'Remaining Debt'])):
        if Debt_Tracker_df.iloc[i,1] > 0:
            Debt_Tracker_df.iloc[i,1] = Debt_Tracker_df.iloc[i,1] - Debt_Tracker_df.iloc[i,3]
            Debt_Tracker_df.iloc[i,2] = Debt_Tracker_df.iloc[i,2] - 1
            paid_debt += Debt_Tracker_df.iloc[i,3]
    
    Debt_Tracker_df.to_csv(directory + '\\3 Debt Info\\' + debt_file_name)

    return paid_debt

In [14]:
#Reads a reference CSV to start a DataFrame
Out_DayX_df_Initial = pd.read_csv(directory + '\\4 Reference CSVs\\Output-Reference.csv')

#Merge Salary Information
Employee_Wages_df = Employee_Information_df.loc[:,["ID Number","Rate Per Day"]]
Out_DayX_df = pd.merge(Employee_Wages_df,Out_DayX_df_Initial,on="ID Number",how="left")

#Input Information
Out_DayX_df.loc[:,"ID Number"] = Day_df.loc[:,"ID Number"]
Out_DayX_df.loc[:,"TI min"] = Day_df.loc[:,"Time In"].apply(lambda x: time_to_minutes(x))
Out_DayX_df.loc[:,"TO min"] = Day_df.loc[:,"Time Out"].apply(lambda x: time_to_minutes(x))

#Basic Pay
Out_DayX_df.loc[:,"Work Hours"] = Out_DayX_df.loc[:,"TO min"].astype(int) - Out_DayX_df.loc[:,"TI min"].astype(int)
Out_DayX_df.loc[:,"Work Hours"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: time_round_up(x))
Out_DayX_df.loc[:,"Basic Salary"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: work_hours(x)) * (Out_DayX_df.loc[:,"Rate Per Day"]/8)

#Overtime Pay
Out_DayX_df.loc[:,"Overtime Hours"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: overtime_hours(x))
Out_DayX_df.loc[:,"Overtime Pay"] = (Out_DayX_df.loc[:,"Overtime Hours"] * (Out_DayX_df.loc[:,"Rate Per Day"]/8) * 1.3).round(2)

#Lates
Standard_Clock_In = Standard_Clock_Hours_df.loc[0:0,"Clock In Time"].apply(lambda x: time_to_minutes(x))
Stringone = str(Standard_Clock_In)
Stringone = re.sub(r"\D",r"",Stringone)
Stringone = int(Stringone)
Out_DayX_df.loc[:,"Late Minutes"] = Out_DayX_df.loc[:,"TI min"].astype(int) - Stringone
Out_DayX_df.loc[:,"Late Hours"] = Out_DayX_df.loc[:,"Late Minutes"].apply(lambda x: late_hours(x))

#Late Deduction
Out_DayX_df.loc[:,"Late Deductions"] = (Out_DayX_df.loc[:,"Late Hours"] * (Out_DayX_df.loc[:,"Rate Per Day"]/8)).round(2)

#Gross Salary
Out_DayX_df.loc[:,"Gross Salary"] = (Out_DayX_df.loc[:,"Basic Salary"] + Out_DayX_df.loc[:,"Overtime Pay"] - Out_DayX_df.loc[:,"Late Deductions"]).round(2)

#Debt
Out_DayX_df.loc[:,"Debt"] = Out_DayX_df.loc[:,"ID Number"].apply(lambda x: debt_tracker(x))

#Miscellaneous
Out_DayX_df.loc[:,"Deductibles"] = Day_df.loc[:,"Deductibles"]
Out_DayX_df.loc[:,"Bonus"] = Day_df.loc[:,"Bonus"]

#Standing Salary
Out_DayX_df.loc[:,"Standing Salary"] = (Out_DayX_df.loc[:,"Gross Salary"] - Out_DayX_df.loc[:,"Debt"] - Out_DayX_df.loc[:,"Deductibles"] + Out_DayX_df.loc[:,"Bonus"]).round(2)

C:\Users\ASUS\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_p

In [15]:
#To check the data
Out_DayX_df

,ID Number,Rate Per Day,TI min,TO min,Work Hours,Basic Salary,Overtime Hours,Overtime Pay,Late Minutes,Late Hours,Late Deductions,Gross Salary,Debt,Deductibles,Bonus,Standing Salary
0,10000000,537,470,960,8,537.0,0,0.00,-10,0,0.00,537.00,0,0,0,537.00
1,10000001,600,480,1004,9,600.0,1,97.50,0,0,0.00,697.50,0,20,0,677.50
2,10000002,840,466,972,8,840.0,0,0.00,-14,0,0.00,840.00,0,45,0,795.00
3,10000003,550,467,960,8,550.0,0,0.00,-13,0,0.00,550.00,0,0,0,550.00
4,10000004,1045,503,1010,8,1045.0,0,0.00,23,1,130.62,914.38,30,0,0,884.38
5,10000005,729,536,1012,8,729.0,0,0.00,56,1,91.12,637.88,0,60,0,577.88
6,10000006,1255,480,1040,9,1255.0,1,203.94,0,0,0.00,1458.94,16,170,0,1272.94
7,10000007,920,473,961,8,920.0,0,0.00,-7,0,0.00,920.00,120,100,0,700.00
8,10000008,620,443,964,9,620.0,1,100.75,-37,0,0.00,720.75,50,20,0,650.75
9,10000009,845,476,960,8,845.0,0,0.00,-4,0,0.00,845.00,110,60,0,675.00


In [18]:
#Outputs a csv called "Out-(Name of Input)"
out_folder_name = "Out-" + folder_name
out_directory_folder_name = directory + "\\" + folder_name + "\\" + out_folder_name + "\\Out-" + day_name + ".csv"

Output_csv = Out_DayX_df.to_csv(out_directory_folder_name)